<a href="https://colab.research.google.com/github/t3r4n/Qu-mica-computacional/blob/Transformaci%C3%B3n-a-grafos/Grafos_nuevo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import rdkit
from rdkit import Chem
from rdkit.Chem import rdDepictor
import networkx as nx
import matplotlib.pyplot as plt
import os

In [ ]:
file_path = r"/content/base_mol2.mol2"
if os.path.exists(file_path):
    print("El archivo existe y está accesible.")
else:
    print("El archivo NO existe o la ruta es incorrecta.")

In [ ]:
file_path = r"/content/base_mol2.mol2"
with open(file_path, "r") as f:
    mol2_block = f.read()
mol = Chem.MolFromMol2Block(mol2_block, sanitize=True, removeHs=False)
if mol:
    print("✅ Archivo .mol2 cargado correctamente en RDKit")
else:
    print("❌ Error al cargar el archivo .mol2 en RDKit")

In [ ]:
import networkx as nx
import plotly.graph_objects as go
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem

# Leer el archivo .mol2 con RDKit
#mol = Chem.MolFromMol2File('ruta/a/tu/archivo.mol2')

# Crear el grafo
G = nx.Graph()

# Agregar nodos (átomos)
for atom in mol.GetAtoms():
    G.add_node(atom.GetIdx(),
               element=atom.GetSymbol(),
               charge=atom.GetFormalCharge())

# Agregar aristas (enlaces)
for bond in mol.GetBonds():
    G.add_edge(bond.GetBeginAtomIdx(), bond.GetEndAtomIdx(),
               order=bond.GetBondTypeAsDouble())  # 1.0 (simple), 2.0 (doble), etc.

# Posiciones 3D con layout de spring (dim=3 para 3D)
pos = nx.spring_layout(G, dim=3, seed=42)  # Usamos NetworkX para obtener un layout en 3D

# Obtener las coordenadas de los nodos (átomos)
xyz = np.array([pos[i] for i in G.nodes()])
node_x, node_y, node_z = xyz[:, 0], xyz[:, 1], xyz[:, 2]

# Crear aristas con colores y grosor proporcional al tipo de enlace
edge_traces = []
for u, v, data in G.edges(data=True):
    x0, y0, z0 = pos[u]
    x1, y1, z1 = pos[v]
    order = int(round(data.get("order", 1.0)))  # Orden del enlace (1, 2, 3)

    # Vector unitario perpendicular para desplazar líneas paralelas (pequeño offset)
    offset = np.array([0.03, 0.03, 0.03])

    # Determinamos el color según el orden del enlace
    if order == 1:
        color = 'gray'  # Enlace simple (gris)
    elif order == 2:
        color = 'red'  # Enlace doble (rojo)
    else:
        color = 'blue'  # Enlace triple (azul)

    for i in range(order):
        # Desplazar un poco si es enlace doble o triple
        disp = (i - (order - 1)/2) * offset if order > 1 else np.array([0, 0, 0])
        start = np.array([x0, y0, z0]) + disp
        end = np.array([x1, y1, z1]) + disp
        edge_traces.append(go.Scatter3d(
            x=[start[0], end[0], None],
            y=[start[1], end[1], None],
            z=[start[2], end[2], None],
            mode='lines',
            line=dict(color=color, width=3),  # Color según el enlace
            hoverinfo='skip'
        ))

# Crear nodos con colores por elemento
element_colors = {
    "H": "white",
    "C": "black",
    "N": "blue",
    "O": "red",
    "F": "green",
    "Cl": "green",
    "Br": "brown",
    "I": "purple",
    "S": "yellow",
    "P": "orange",
}

node_trace = go.Scatter3d(
    x=node_x, y=node_y, z=node_z,
    mode='markers+text',
    marker=dict(
        size=6,
        color=[element_colors.get(G.nodes[i]["element"], "pink") for i in G.nodes()],
        line=dict(width=1, color='black')
    ),
    text=[G.nodes[i]['element'] for i in G.nodes()],
    textposition='top center',
    hovertext=[
        f"Átomo: {G.nodes[i]['element']}<br>Carga: {G.nodes[i]['charge']}"
        for i in G.nodes()
    ],
    hoverinfo='text'
)

# Crear figura
fig = go.Figure(data=edge_traces + [node_trace],
                layout=go.Layout(
                    title="🔬 Grafo Molecular 3D (colores por enlace y átomo)",
                    showlegend=False,
                    margin=dict(l=0, r=0, b=0, t=40),
                    scene=dict(
                        xaxis=dict(visible=False),
                        yaxis=dict(visible=False),
                        zaxis=dict(visible=False)
                    )
                ))

fig.show()
